### Trabalho Prático 2 
__Nome:__ Gabriela Tavares Barreto 

__Matrícula:__ 2018074657

## Código

### Importando módulos 

In [1]:
import numpy as np
import networkx as nx
import heapq as hq
import matplotlib.pyplot as plt
import sys
import math
import time
import os

INF = float(sys.maxsize)

### Funções de leitura de dados

In [2]:
def ler_arquivo(arq):
    arquivo = open(arq, 'r')
    vertices = []
    flag = False
    for linha in arquivo:
        if not flag:
            if linha.split()[0] == 'DIMENSION:':
                d = int(linha.split()[1])
            elif linha.split()[0] == 'DIMENSION':
                d = int(linha.split()[2])
            if linha == 'NODE_COORD_SECTION\n':
                flag = True
                continue
            
        if linha == 'EOF\n' or linha == 'EOF' or linha == '\n':
            break

        if flag:
            vertice = [0, (0,0)]
            lista_num = linha.split()
            vertice[0] = float(lista_num[0])
            vertice[1] = (float(lista_num[1]), float(lista_num[2]))
            vertices.append(vertice)
    
    arquivo.close()

    return d, vertices

### Funções auxiliares

In [3]:
# funções auxiliares
def distancia_euclidiana(a, b):
    return np.sqrt((a[0]-b[0])**2+(a[1]-b[1])**2)

def calcula_arestas(d, dados):
    arestas = []
    for i in range(0, d-1):
        for j in range(i+1, d):
            aresta = [0, 0, 0]
            aresta[0] = dados[i][0]  # vertice 1
            aresta[1] = dados[j][0]  # vertice 2
            aresta[2] = distancia_euclidiana(dados[i][1], dados[j][1])  # peso
            arestas.append(aresta)
    return arestas

def custo_circuito(c, G, n):
    custo = 0
    for i in range(-1,n-2):
        custo += G[c[i]][c[i+1]]['weight']
    
    return custo

### Branch and bound

In [4]:
class Node:
    def __init__(self, bound, caminho, nivel):
        self.bound = bound
        #self.lista = lista
        self.caminho = caminho
        self.nivel = nivel

    def __lt__(self, outro):
        return self.bound < outro.bound
    
    
def no_caminho(c, v):
    for i in c:
        if i == v:
            return True
    return False
        

# calcula limite inferior da raiz
def bound_raiz(arestas, n):
    s = 0
    for i in range(1, n+1):
        s += arestas[i][0] + arestas[i][1]
    return math.ceil(s/2)

def bound_ultimo(c, j, g, n):
    s = 0
    for i in range(1, n-2):
        s += g[c[i-1]][c[i]]['weight'] +  g[c[i]][c[i+1]]['weight']
    s += g[c[n-3]][c[n-2]]['weight']+ g[c[n-2]][j]['weight']
    s += g[c[n-2]][j]['weight'] + 2*g[j][c[1]]['weight'] + g[1][c[1]]['weight']

    return math.ceil(s/2)

def menores_arestas(g, n):  # pega duas menores arestas de cada vertice e retorna em uma lista 
    arestas = np.empty((n+1, 2))
    # Obter as arestas conectadas a esse vértice, juntamente com os pesos
    for i in range(1, n+1):
        pesos = [(g[i][j]['weight']) for j in g[i]]
        # Ordenar as arestas pelo peso
        pesos.sort(key=lambda x: x)
        # Obter as duas arestas com menores pesos
        arestas[i][0] = pesos[0]
        arestas[i][1] = pesos[1]  
    return arestas

def bound(c, j, menores_arestas, n, g):
    aux = bound = 0

    if c[1] == 0: # quando vai preencher a segunda posição
        aux = g[1][j]['weight']
    else:
        aux = g[1][c[1]]['weight']

    # se arestas de 1 pro proximo nao for uma das menores arestas
    if aux != menores_arestas[1][0] and aux != menores_arestas[1][1]:
        bound = aux + menores_arestas[1][0]
    else:
        bound = menores_arestas[1][0] + menores_arestas[1][1]
    
    for i in range(1, n):
        if c[i+1] == 0:
            #print(c[i],i)
            bound += g[c[i-1]][c[i]]['weight']+ 2*g[c[i]][j]['weight']
            if g[c[i]][j]['weight'] == menores_arestas[j][0]:
                bound += menores_arestas[j][1]
            else: 
                bound += menores_arestas[j][0]
            break
        else:
            bound += g[c[i-1]][c[i]]['weight']+ g[c[i]][c[i+1]]['weight']

    nao_visitado = set(g.nodes) - set(c) - {j}
    for i in nao_visitado:
        bound += menores_arestas[i][0] + menores_arestas[i][1]

    return math.ceil(bound/2)



def branch_and_bound(G, N):
    c = np.zeros(N, dtype=int) #caminho vazio
    c[0] = 1
    arestas = menores_arestas(G, N)
    b = bound_raiz(arestas, N) # limite inferior da raiz
    raiz = Node(b, c, 1) # bound, caminho, nível da arvore
   
    h = []
    hq.heappush(h, raiz)
    melhor = INF
    sol = []

    while(len(h) != 0):
        node = hq.heappop(h)
        if node.nivel == N:
            custo_sol = custo_circuito(node.caminho, G, N)
            if melhor > custo_sol:
                melhor = custo_sol
                sol = node.caminho
        elif node.bound < melhor:
            if node.nivel < N:
                for j in range(1, N+1):
                    #if not no_caminho(node.caminho, j):
                    if j not in set(node.caminho):
                        novo_bound = bound(node.caminho, j, arestas, node.nivel, G)
                        if novo_bound < melhor:
                            c_novo = node.caminho.copy()
                            c_novo[node.nivel] = j
                            
                            #aux_node = Node(novo_bound, c_novo, node.nivel+1)
                            hq.heappush(h, Node(novo_bound, c_novo, node.nivel+1))
            #if node.nivel < N-1:
    return melhor, sol


### Twice around the three & Christofides

In [5]:
def twice_around_the_three(grafo, n):
    T = nx.minimum_spanning_tree(grafo)
    caminho = np.array(list(nx.dfs_preorder_nodes(T, source=2)))
    custo = custo_circuito(caminho, grafo, n)
    return custo, caminho

def christofides(grafo, n):
    T = nx.minimum_spanning_tree(grafo) 
    vertices = np.array(list(grafo)) # todos vertices
    grau = list(T.degree(vertices)) #lista de vertices e respectivos graus
    grau_impar = [x[0] for x in grau if x[1]%2 == 1] 

    H = nx.induced_subgraph(grafo, grau_impar)
    H = nx.Graph(H)
    H.remove_nodes_from([n for n in H if n not in set(grau_impar)])
    conj_arestas = nx.min_weight_matching(H)

    M = nx.Graph()
    for u, v in conj_arestas:
        if grafo.has_edge(u, v):
            peso = grafo[u][v]['weight']
            M.add_edge(u, v, weight=peso)
    C = nx.compose(T, M)

    caminho = np.array(list(nx.dfs_preorder_nodes(C, source=1)))
    custo = custo_circuito(caminho, grafo, n)
    
    return custo, caminho

### Função para testar instâncias:

In [6]:
def experimento(caminho_pasta):
    for arquivo in os.listdir(caminho_pasta):
        if arquivo[-3:] == 'tsp': 
            caminho_completo = os.path.join(caminho_pasta, arquivo)
            dim, dados = ler_arquivo(caminho_completo)

            if dim > 4000:
                continue

            arestas = calcula_arestas(dim, dados)
            grafo = nx.Graph()
            grafo.add_weighted_edges_from(arestas)

            start_twice = time.time()
            sol_twice, rota_twice = twice_around_the_three(grafo, dim)
            t_twice = time.time() - start_twice

            if dim < 4000:
                start_chris = time.time()
                sol_chris, rota_chris = christofides(grafo, dim)
                t_chris = time.time() - start_chris
            else:
                start_chris = sol_chris = t_chris = -1 

            dataset = arquivo[:-4]
            a = open('tp2_datasets.txt')
            for linha in a:
                for palavra in linha.split():
                    if palavra == dataset:
                        sol_opt = linha.split()[2]

            with open('resultados.txt', 'a') as arq:
                arq.write(f'arq: {arquivo}, Vértices: {dim}, Sol.chris: {math.ceil(sol_chris)}, Sol.twice: {math.ceil(sol_twice)}, Sol.opt: {sol_opt}, t_chris: {t_chris}, t_twice: {t_twice} \n')
                arq.close()

## Experimentos

Obs: forçei o encerramento do experimento.

In [7]:
experimento('./instancias/')

KeyboardInterrupt: 